`Designed for Brightway2` \
`v1` \
`13-04-2023` \
`Authors: Michael Weinold (@michaelweinold) and Chris Mutel (@cmutel)`

# Imports

In [ ]:
import zipfile
import os
import stats_arrays

In [ ]:
import bw2data
import bw2io
import bw2calc
import bw2analyzer

In [ ]:
# type hints
from bw2data.backends.peewee.database import SQLiteBackend
from bw2data.backends.peewee.proxies import Activity
from bw2calc.lca import LCA

# Projects

In Brightway2, a project is a separate directory with its own copies of LCI databases, LCIA methods, and any other data you use. Each research project or article should probably be its own project, so that any changes you want to make will not interfere with your other work. The default project is called ``default``:

In [ ]:
print(bw2data.projects.current)
print(bw2data.projects.dir)

We can create a new project:

In [ ]:
bw2data.projects.set_current("bw2intro")

And list the available projects:

In [ ]:
bw2data.projects

Projects can be deleted:

In [ ]:
bw2data.projects.delete_project(name = 'default', delete_dir = True)

# Biosphere Dataset

The `bw2io.bw2setup()` function can import some basic data: a database of elementary flows, some LCIA methods, and some metadata used for importing other databases.

In [ ]:
bw2io.bw2setup()

In [ ]:
db: SQLiteBackend = bw2data.Database("biosphere3")

The number of biosphere flows is:

In [ ]:
len(db)

We can get a random flow and the associated name, unit and categories:

In [ ]:
random_flow: Activity = db.random()

In [ ]:
random_flow

In [ ]:
print(random_flow['name'])
print(random_flow['unit'])
print(random_flow['categories'])

Brightway2 uses keys to identify datasets. Each dataset is identified by a combination of its database and some unique code. The code can be anything - a number, a UUID, or just a name.

In [ ]:
random_flow.key

# LCIA Methods Dataset

The `bw2io.bw2setup()` function also installed a large number of LCIA methods:

In [ ]:
bw2data.methods

The number of methods is:

In [ ]:
len(bw2data.methods)

Because LCIA methods have many different impact categories, they are identified not by a single label, but by a list of labels. Let's look at an example:

In [ ]:
method_key: tuple = bw2data.methods.random()
method_key

In this case, the LCIA method has three levels of specificity, from the general name (first level) to the specific impact category (last level). There is nothing magic about three levels - you could have one, or one hundred - but Brightway2 expects that LCIA methods will be a list of text labels ``('like', 'this')``.

We can load the method data, show a sample.

Method data has the format:

    biosphere flow, numeric value, location

Where:

* `biosphere flow` is a dataset from any database which is used as a biosphere flow.
* `numeric value` can be either a fixed number or an uncertainty distribution.
* `location` is optional; the default value is that this characterization factor is valid everywhere.

The method data format is pretty flexible, and the following are all acceptable:

    [('biosphere', 'CO2'), 1.0],                                             # Default location
    [('biosphere', 'CO2'), 1.0, 'Australia, mate!'],                         # Custom location
    [('biosphere', 'CO2'), 1.0, ('Population density', 'Raster cell 4,2')],  # Location inside a geocollection
    [('biosphere', 'CO2'), {'amount': 1.0, 'uncertainty type': 0}],          # Uncertain characterization factor

[Geocollections](http://brightway2-regional.readthedocs.org/#spatial-scales-geocollections) are needed for regionalized LCA.

If you are wondering why we need to identify biosphere flows like `('biosphere', '2fe885840cebfcc7d56b607b0acd9359')`, this is a good question! The short answer is that there is no single field that uniquely identifies biosphere flows or activities. The longer answer [is in the manual](https://docs.brightwaylca.org/intro.html#uniquely-identifying-activities).

Brightway2 is designed to be flexible enough for many different problems. Therefore, there are no limits on what constitutes a biosphere flow. Rather, anything that is linked to in a biosphere exchange will be put in the biosphere matrix. We installed a database called `biosphere3`, but you can define new flows in a database alongside process datasets, or create your own biosphere database.

In [ ]:
method_data = Method(method_key).load()
print("Number of CFs:", len(method_data))
method_data[:20]

# Importing the `FORWAST` LCI Database

We will use the FORWAST database, as it is both a high quality, comprehensive LCI database, and freely available. [FORWAST](http://forwast.brgm.fr/Overview.asp) is a physical MRIO table for Europe. It can be downloaded directly from the [2.-0 website](http://lca-net.com/projects/show/forwast/).

In [ ]:
filepath = bw2data.utils.download_file("forwast.bw2package.zip", url="http://lca-net.com/wp-content/uploads/")
dirpath = os.path.dirname(filepath)
zipfile.ZipFile(filepath).extractall(dirpath)
bw2io.BW2Package.import_file(os.path.join(dirpath, "forwast.bw2package"))

In [ ]:
bw2data.databases

# Searching Datasets

The `bw2data.Database.search` function covers the following data fields:
* `name`
* `comment`
* `product`
* `categories`
* `location`

In [ ]:
bw2data.Database("forwast").search("food")

You can also use the `*` wildcard in search queries. By default, the function returns the first 25 search results.

In [ ]:
bw2data.Database("biosphere3").search("carb*", limit=10)

Searches can also be filtered (where only the results that meet the specified criteria are *included*) or masked (where results that meet the specified criteria are *excluded*). You can specify inclusion or exclusion criteria for fields with `filter` and `mask`:

In [ ]:
bw2data.Database("biosphere3").search("carbon", filter={"categories": 'forestry'})

In [ ]:
bw2data.Database("biosphere3").search("carbon", limit=10, mask={"categories": 'forestry'})

We can sort by `location`, `name`, `product` (reference product), or `type`, by specifying the `order_by` property.

In [ ]:
db.order_by = "location"
print_10(db)

Set `.order_by = None` to remove any ordering.

In [ ]:
db.order_by = None

Because accessing activities in the database is quite fast, you can also filter the activities you want by just iterating over the entire database:

In [ ]:
my_activities = [x for x in db if 'Electr' in x['name']]
my_activities

The corresponding query using the `search` function is:

In [ ]:
bw2data.Database('forwast').search('electr*')

# Basic LCA Calculations

In matrix-based LCA, we construct a *technosphere* matrix, which describes the inputs needed to produce different products (e.g. cars need metal and electricity), and a *biosphere* matrix, which describes the emissions and resource consumption associated with the production of each product (e.g. car manufacturing releases air emissions). These two matrices come from the life cycle inventory database(s). We also have a *functional unit*, which is what we are trying to assess, e.g. one car. We then calculate the *life cycle inventory* (LCI) by first solving the linear system of the technosphere matrix and the functional unit, and then by multiplying the biosphere matrix. To do *life cycle impact assessment* (LCIA), we multiply the life cycle inventory by a matrix of characterization factors, which tell how bad different emissions and resource consumptions are.

We can pick a random process and LCIA method:

In [ ]:
process: Activity = Database("forwast").random()
process

We now use it to specify the functional unit of the calculation:

In [ ]:
functional_unit: dict = {process: 1}

We can then instantiate our [LCA object](https://docs.brightwaylca.org/technical/bw2calc.html#lca). 

In [ ]:
lca: LCA = bw2calc.LCA(demand = functional_unit, method = method_key)

And do the LCI and LCIA calculations:

In [ ]:
lca.lci()
lca.lcia()

Finally, we can print the LCA score:

In [ ]:
lca.score

We can reuse the same matrices but change the functional unit by using the `redo_lci` and `redo_lcia` functions:

In [ ]:
new_process: Activity = Database("forwast").random()
print(new_process)
lca.redo_lcia({new_process: 1})
lca.score

## Looking into the LCA Object

Let's see what is in this `LCA` thing, anyway. Let's look at a few things:

 - The technosphere matrix

In [ ]:
lca.technosphere_matrix

* The biosphere matrix

In [ ]:
lca.biosphere_matrix

* The characterization matrix

In [ ]:
lca.characterization_matrix

# Contribution Analysis

We can calculate the most important activities and biosphere flows. First, the most important activities:

In [ ]:
bw2analyzer.ContributionAnalysis().annotated_top_processes(lca)

Now the most important biosphere flows:

In [ ]:
bw2analyzer.ContributionAnalysis().annotated_top_emissions(lca)

# Monte Carlo LCA

Unfortunately, the `forwast` database doesn't unclude uncertainty. Let's put some in anyways, using the utility function [uncertainify](https://docs.brightwaylca.org/technical/bw2data.html#bw2data.utils.uncertainify).

In [ ]:
uncertain_db = Database("forwast uncertain +")
uncertain_db.write(
    uncertain_db.relabel_data(
        bw2data.utils.uncertainify(
            Database("forwast").load(), 
            stats_arrays.NormalUncertainty
        ), 
        "forwast uncertain +" 
    )
)

We can now calculate some Monte Carlo iterations for a random activity.

In [ ]:
mc = MonteCarloLCA(demand={uncertain_db.random(): 1}, method=method_key)
mc.load_data()
for x in range(10):
    print(next(mc))